In [1]:
using JuMP
using Gurobi
using CSV
using DataFrames

In [2]:
d = CSV.read("wnba_distances.csv", DataFrame)[:, 2:end]

teams = names(d)
n = length(teams)

conferences = Dict(
    "Eastern" => ["Atlanta Dream", "Chicago Sky", "Connecticut Sun", "Indiana Fever", 
        "New York Liberty", "Washington Mystics"],
    "Western" => ["Dallas Wings", "Las Vegas Aces", 
        "Los Angeles Sparks", "Minnesota Lynx", "Phoenix Mercury", "Seattle Storm"]
)

team_indices = Dict(team => i for (i, team) in enumerate(teams))
indices_team = Dict(i => team for (i, team) in enumerate(teams))
conference_indices = Dict(conf => [team_indices[team] for team in teams] for (conf, teams) in conferences)

Dict{String, Vector{Int64}} with 2 entries:
  "Eastern" => [1, 2, 3, 4, 5, 6]
  "Western" => [7, 8, 9, 10, 11, 12]

In [10]:
# no road trips

model = Model(Gurobi.Optimizer)

# one away variable for each team (away team, home team, day)
@variable(model, a[1:12, 1:12, 1:128], Bin)

# each team plays 20 home games 20 away games
@constraint(model, [i=1:12], sum(a[i,j,k] for j in 1:12, k in 1:128) == 20)

# two games cannot be played on the same day
@constraint(model, [i=1:12, k=1:128], sum(a[i,j,k] + a[j,i,k] for j in 1:12) <= 1)

# no games against self
@constraint(model, [i=1:12, k=1:128], a[i,i,k] == 0)

# fairness constraint: teams with the most travel distance - all teams travel <= 40000 mi
@constraint(model, [i=1:12], sum(d[i,j]*a[i,j,k] for j in 1:12, k in 1:128) <= 40000)

# each team must play 4 games against each team in conference (2 home 2 away)

for (conference, _) in conferences
    for i in conference_indices[conference]
        for j in conference_indices[conference] if j != i
            @constraint(model,
                sum(a[i,j,k] for k in 1:128) == 2
            )
        end
        end
    end
end

# each team must play at least 3 games against out of conference opponents

# this allows algorithm to assign 4 teams that are out of conference to
# play 4 games against instead of 3.

for i in conference_indices["Eastern"]
    for j in conference_indices["Western"]
        @constraint(model,
            sum(a[i,j,k] + a[j,i,k] for k in 1:128) >= 3
            )
    end
end

# each team must play at most 4 games against out of conference opponents


for i in conference_indices["Eastern"]
    for j in conference_indices["Western"]
        @constraint(model,
            sum(a[i,j,k] + a[j,i,k] for k in 1:128) <= 4
            )
    end
end


set_optimizer_attribute(model, "NodefileStart", 200) 
set_optimizer_attribute(model, "Presolve", 1)
set_optimizer_attribute(model, "Threads", 16)  # Use all CPU cores
set_optimizer_attribute(model, "IISMethod", 1)  # Use IIS analysis
set_optimizer_attribute(model, "OutputFlag", 1)  # Enable detailed log output
set_optimizer_attribute(model, "TimeLimit", 1.00e10)

@objective(model, Min, 2*sum(d[i,j]*a[i,j,k] for i in 1:12, j in 1:12, k in 1:128))

optimize!(model)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-04
Set parameter NodefileStart to value 200
Set parameter Presolve to value 1
Set parameter Threads to value 16
Set parameter IISMethod to value 1
Set parameter TimeLimit to value 10000000000
Set parameter IISMethod to value 1
Set parameter Threads to value 16
Set parameter NodefileStart to value 200
Set parameter TimeLimit to value 10000000000
Set parameter Presolve to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 6900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3228 rows, 18432 columns and 98304 nonzeros
Model fingerprint: 0x1a932ff8
Variable types: 0 continuous, 18432 integer (18432 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [2e+02, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RH

In [11]:
# Results
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.TIME_LIMIT
    println("Optimal or time-limited solution found.")
    for k in 1:128
        for i in 1:12
            for j in 1:12
                if value(a[i, j, k]) > 0.5
                    println("Day $k: $(teams[i]) plays at $(teams[j])")
                end
            end
        end
    end
else
    println("No feasible solution found.")
end

Optimal or time-limited solution found.
Day 1: Chicago Sky plays at Las Vegas Aces
Day 1: Indiana Fever plays at Seattle Storm
Day 1: Los Angeles Sparks plays at New York Liberty
Day 1: Minnesota Lynx plays at Atlanta Dream
Day 1: Phoenix Mercury plays at Connecticut Sun
Day 2: Chicago Sky plays at Las Vegas Aces
Day 2: Dallas Wings plays at New York Liberty
Day 2: Los Angeles Sparks plays at Atlanta Dream
Day 2: Phoenix Mercury plays at Connecticut Sun
Day 2: Seattle Storm plays at Washington Mystics
Day 3: Atlanta Dream plays at Phoenix Mercury
Day 3: Chicago Sky plays at Las Vegas Aces
Day 3: Indiana Fever plays at Los Angeles Sparks
Day 3: Seattle Storm plays at Connecticut Sun
Day 4: Dallas Wings plays at Los Angeles Sparks
Day 5: Atlanta Dream plays at Washington Mystics
Day 5: New York Liberty plays at Connecticut Sun
Day 6: Las Vegas Aces plays at New York Liberty
Day 6: Minnesota Lynx plays at Washington Mystics
Day 8: Chicago Sky plays at Indiana Fever
Day 8: Los Angeles Spar

In [24]:
for i in 1:12
    println("$(teams[i]) traveled $(2*sum(d[i,j]*value(a[i,j,k]) for j in 1:12, k in 1:128)) miles")
end

using Statistics

distances = [2*sum(d[i,j]*value(a[i,j,k]) for j in 1:12, k in 1:128) for i in 1:12]
println(sum(distances))
println(mean(distances))

Atlanta Dream traveled 40898.65180295262 miles
Chicago Sky traveled 40120.54326115973 miles
Connecticut Sun traveled 48143.568967834246 miles
Indiana Fever traveled 32639.225421010666 miles
New York Liberty traveled 43298.81406288185 miles
Washington Mystics traveled 40438.36227430953 miles
Dallas Wings traveled 43386.633159711215 miles
Las Vegas Aces traveled 55458.984783000335 miles
Los Angeles Sparks traveled 58866.667104218424 miles
Minnesota Lynx traveled 38893.10626511654 miles
Phoenix Mercury traveled 52587.93203639518 miles
Seattle Storm traveled 67721.76371524195 miles
562454.2528538323
46871.18773781936


In [32]:
# no road trips

model = Model(Gurobi.Optimizer)

# one away variable for each team (away team, home team, day)
@variable(model, a[1:12, 1:12, 1:128], Bin)

# each team plays 20 home games 20 away games
@constraint(model, [i=1:12], sum(a[i,j,k] for j in 1:12, k in 1:128) == 19)

# two games cannot be played on the same day
@constraint(model, [i=1:12, k=1:128], sum(a[i,j,k] + a[j,i,k] for j in 1:12) <= 1)

# no games against self
@constraint(model, [i=1:12, k=1:128], a[i,i,k] == 0)

# fairness constraint: teams with the most travel distance - all teams travel <= 40000 mi
@constraint(model, [i=1:12], sum(d[i,j]*a[i,j,k] for j in 1:12, k in 1:128) <= 40000)

# each team must play 4 games against each team in conference (2 home 2 away)

for (conference, _) in conferences
    for i in conference_indices[conference]
        for j in conference_indices[conference] if j != i
            @constraint(model,
                sum(a[i,j,k] for k in 1:128) == 2
            )
        end
        end
    end
end

# each team must play at least 3 games against out of conference opponents

# this allows algorithm to assign 4 teams that are out of conference to
# play 4 games against instead of 3.

for i in conference_indices["Eastern"]
    for j in conference_indices["Western"]
        @constraint(model,
            sum(a[i,j,k] + a[j,i,k] for k in 1:128) >= 3
            )
    end
end

# each team must play at most 4 games against out of conference opponents


for i in conference_indices["Eastern"]
    for j in conference_indices["Western"]
        @constraint(model,
            sum(a[i,j,k] + a[j,i,k] for k in 1:128) <= 4
            )
    end
end


set_optimizer_attribute(model, "NodefileStart", 200) 
set_optimizer_attribute(model, "Presolve", 1)
set_optimizer_attribute(model, "Threads", 16)  # Use all CPU cores
set_optimizer_attribute(model, "IISMethod", 1)  # Use IIS analysis
set_optimizer_attribute(model, "OutputFlag", 1)  # Enable detailed log output
set_optimizer_attribute(model, "TimeLimit", 1.00e10)

@objective(model, Min, 2*sum(d[i,j]*a[i,j,k] for i in 1:12, j in 1:12, k in 1:128))

optimize!(model)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-04
Set parameter NodefileStart to value 200
Set parameter Presolve to value 1
Set parameter Threads to value 16
Set parameter IISMethod to value 1
Set parameter TimeLimit to value 10000000000
Set parameter IISMethod to value 1
Set parameter Threads to value 16
Set parameter NodefileStart to value 200
Set parameter TimeLimit to value 10000000000
Set parameter Presolve to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 6900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3228 rows, 18432 columns and 98304 nonzeros
Model fingerprint: 0xbdd5d4bd
Variable types: 0 continuous, 18432 integer (18432 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [2e+02, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RH

In [33]:
# Results
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.TIME_LIMIT
    println("Optimal or time-limited solution found.")
    for k in 1:128
        for i in 1:12
            for j in 1:12
                if value(a[i, j, k]) > 0.5
                    println("Day $k: $(teams[i]) plays at $(teams[j])")
                end
            end
        end
    end
else
    println("No feasible solution found.")
end

Optimal or time-limited solution found.
Day 1: Indiana Fever plays at Los Angeles Sparks
Day 1: Minnesota Lynx plays at Washington Mystics
Day 3: Chicago Sky plays at Las Vegas Aces
Day 3: Dallas Wings plays at Indiana Fever
Day 4: Los Angeles Sparks plays at Dallas Wings
Day 4: Minnesota Lynx plays at New York Liberty
Day 5: Connecticut Sun plays at Seattle Storm
Day 5: Washington Mystics plays at New York Liberty
Day 5: Las Vegas Aces plays at Dallas Wings
Day 6: Chicago Sky plays at Phoenix Mercury
Day 6: Las Vegas Aces plays at New York Liberty
Day 6: Minnesota Lynx plays at Los Angeles Sparks
Day 8: Connecticut Sun plays at Indiana Fever
Day 9: Atlanta Dream plays at Phoenix Mercury
Day 10: Minnesota Lynx plays at Dallas Wings
Day 11: Washington Mystics plays at Atlanta Dream
Day 11: Minnesota Lynx plays at Indiana Fever
Day 12: Las Vegas Aces plays at Washington Mystics
Day 13: Phoenix Mercury plays at Washington Mystics
Day 14: Atlanta Dream plays at Los Angeles Sparks
Day 14: C

In [34]:
for i in 1:12
    println("$(teams[i]) traveled $(2*sum(d[i,j]*value(a[i,j,k]) for j in 1:12, k in 1:128)) miles")
end

using Statistics

distances = [2*sum(d[i,j]*value(a[i,j,k]) for j in 1:12, k in 1:128) for i in 1:12]
println(sum(distances))
println(mean(distances))

Atlanta Dream traveled 46400.148612315694 miles
Chicago Sky traveled 34330.063437506964 miles
Connecticut Sun traveled 43337.878389114805 miles
Indiana Fever traveled 34295.39253474161 miles
New York Liberty traveled 41655.78165559827 miles
Washington Mystics traveled 41589.67622283482 miles
Dallas Wings traveled 40066.61661065476 miles
Las Vegas Aces traveled 50345.1836173511 miles
Los Angeles Sparks traveled 59300.25424717049 miles
Minnesota Lynx traveled 39150.85109613312 miles
Phoenix Mercury traveled 48717.10015768747 miles
Seattle Storm traveled 60871.5864514905 miles
540060.5330325996
45005.0444193833


In [ ]:
# # for one team only

# n = 6 # number of teams
# T = 40 # number of days
# i = 12 # index of team

# model = Model(Gurobi.Optimizer)

# # one home variable for team (day)
# @variable(model, h[1:T], Bin)

# # one away variable for team (home team, day)
# @variable(model, a[7:12, 1:T], Bin)

# # one location variable for each team to track where they are each day
# @variable(model, loc[7:12, 0:T], Bin)

# # variable to ensure alternation (number of home games in a row)
# @variable(model, y[1:T] >= 0, Int)

# # variable to ensure alternation (number of away games in a row)
# @variable(model, z[1:T] >= 0, Int)

# # each team plays 10 home games 10 away games
# @constraint(model, sum(h[k] for k in 1:T) == 10)
# @constraint(model, sum(a[j,k] for j in 7:12, k in 1:T) == 10)

# # two games cannot be played on the same day
# @constraint(model, [k=1:T], sum(a[j,k] for j in 7:12) + h[k] <= 1)

# # team starts at their own location
# @constraint(model, loc[i,0] == 1)

# # team cannot play away game against self
# @constraint(model, [k=1:T], a[i,k] == 0)

# # team is in exactly one location at a time
# @constraint(model, [k=0:T], sum(loc[j,k] for j in 7:12) == 1)

# # teams must be at correct away location to play away game
# @constraint(model, [j=7:12, k=1:T], a[j,k] <= loc[j,k])

# # teams must be at home to play home game
# @constraint(model, [k=1:T], h[k] <= loc[i,k])

# # team cannot stay in other city for more than 1 day
# @constraint(model, [j=7:11, k=1:T-1], loc[j,k] + loc[j,k+1] <= 1)

# # Initialize the streaks
# @constraint(model, y[1] == h[1])
# @constraint(model, z[1] == sum(a[j, 1] for j in 7:12))

# # Update streaks for each day
# @constraint(model, [k=2:T], y[k] <= (y[k-1] + h[k]) * (1 - sum(a[j,k] for j in 7:12)))
# @constraint(model, [k=2:T], y[k] >= (y[k-1] + h[k]) * (1 - sum(a[j,k] for j in 7:12)))

# @constraint(model, [k=2:T], z[k] <= (z[k-1] + sum(a[j,k] for j in 7:12)) * (1 - h[k]))
# @constraint(model, [k=2:T], z[k] >= (z[k-1] + sum(a[j,k] for j in 7:12)) * (1 - h[k]))

# # Limit the streaks to at most 2
# @constraint(model, [k=1:T], y[k] <= 2)
# @constraint(model, [k=1:T], z[k] <= 2)

# # team must play every other team in conference twice
# for j in 7:12
#     if j != i
#         @constraint(model, sum(a[j,k] for k in 1:T) == 2)
#     end
# end

# set_optimizer_attribute(model, "NodefileStart", 200) # use all memory
# set_optimizer_attribute(model, "Presolve", 2) # aggressive presolve
# set_optimizer_attribute(model, "Threads", 16)  # Use all CPU cores
# set_optimizer_attribute(model, "IISMethod", 1)  # Use IIS analysis
# set_optimizer_attribute(model, "OutputFlag", 1)  # Enable detailed log output
# set_optimizer_attribute(model, "TimeLimit", 1.00e10)

# @objective(model, Min, sum(d[j1,j2]*loc[j1,k-1]*loc[j2,k] for j1 in 7:12, j2 in 7:12, k in 1:T))

# optimize!(model)

# if termination_status(model) == MOI.OPTIMAL
#     println("Optimal travel distances: ", value.(d))
#     println("Schedule: ", value.(a))
# else
#     println("No optimal solution found.")
#     println(termination_status(model))
# end

In [ ]:
# model = Model(Gurobi.Optimizer)

# n = 6 # number of teams
# T = 40 # number of days


# # one away variable for each team (away team, home team, day)
# @variable(model, a[1:n, 1:n, 1:T], Bin)

# # one location variable for each team to track where they are each day
# @variable(model, loc[1:n, 1:n, 0:T], Bin)

# # each team plays 20 home games 20 away games
# @constraint(model, [i=1:n], sum(a[i,j,k] for j in 1:n, k in 1:T) == 10)

# # two games cannot be played on the same day
# @constraint(model, [i=1:n, k=1:T], sum(a[i,j,k] + a[j,i,k] for j in 1:n) <= 1)

# # no games against self
# @constraint(model, [i=1:n, k=1:T], a[i,i,k] == 0)

# # every team starts at their own location
# @constraint(model, [i=1:n], loc[i,i,0] == 1)

# # every team is in exactly one location at a time
# @constraint(model, [i=1:n, k=0:T], sum(loc[i,j,k] for j in 1:n) == 1)

# # teams must be at correct away location to play away game
# @constraint(model, [i=1:n, j=1:n, k=1:T], a[i,j,k] <= loc[i,j,k])

# # teams must be at home to play home game
# @constraint(model, [i=1:n, j=1:n, k=1:T], a[j,i,k] <= loc[i,i,k])

# # teams can only travel somewhere if they have a game in that city
# # @constraint(model, [i=1:n, j1=1:n, j2=1:n, k=1:T],
# #     loc[i, j2, k] <= loc[i, j1, k-1] + sum(a[i, j2, k] + a[j2, i, k] for j2 in 1:n if j2 != j1)
# # )

# # fairness constraint: teams with the most travel distance - team with least travel distance <= 10000 mi



# # each team must play 4 games against each team in conference (2 home 2 away)

# # for (conference, _) in conferences
# #     for i in conference_indices[conference]
# #         for j in conference_indices[conference] if j != i
# #             @constraint(model,
# #                 sum(a[i,j,k] for k in 1:T) == 2
# #             )
# #         end
# #         end
# #     end
# # end

# for i in 1:n
#     for j in 1:n
#         if i != j
#             @constraint(model, sum(a[i,j,k] for k in 1:T) == 2)
#         end
#     end
# end

# # each team must play at least 3 games against out of conference opponents

# # this allows algorithm to assign 4 teams that are out of conference to
# # play 4 games against instead of 3.

# # for i in conference_indices["Eastern"]
# #     for j in conference_indices["Western"]
# #         @constraint(model,
# #             sum(a[i,j,k] + a[j,i,k] for k in 1:T) >= 3
# #             )
# #     end
# # end



# set_optimizer_attribute(model, "NodefileStart", 200) # use all memory
# set_optimizer_attribute(model, "Presolve", 2) # aggressive presolve
# set_optimizer_attribute(model, "Threads", 16)  # Use all CPU cores
# set_optimizer_attribute(model, "IISMethod", 1)  # Use IIS analysis
# set_optimizer_attribute(model, "OutputFlag", 1)  # Enable detailed log output
# set_optimizer_attribute(model, "TimeLimit", 1.00e10)

# @objective(model, Min, sum(d[j1,j2]*loc[i,j1,k-1]*loc[i,j2,k] for i in 1:n, j1 in 1:n, j2 in 1:n, k in 1:T))

# optimize!(model)

# if termination_status(model) == MOI.OPTIMAL
#     println("Optimal travel distances: ", value.(d))
#     println("Schedule: ", value.(a))
# else
#     println("No optimal solution found.")
#     println(termination_status(model))
# end

In [5]:
# Results
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.TIME_LIMIT
    println("Optimal or time-limited solution found.")
    for k in 1:T
        for i in 1:n
            for j in 1:n
                if value(a[i, j, k]) > 0.5
                    println("Day $k: $(teams[i]) plays at $(teams[j])")
                end
            end
        end
    end
else
    println("No feasible solution found.")
end

Optimal or time-limited solution found.
Day 6: Chicago Sky plays at Indiana Fever
Day 8: Chicago Sky plays at Indiana Fever
Day 9: Chicago Sky plays at Indiana Fever
Day 9: New York Liberty plays at Connecticut Sun
Day 10: Chicago Sky plays at Indiana Fever
Day 10: New York Liberty plays at Connecticut Sun
Day 11: Chicago Sky plays at Indiana Fever
Day 11: New York Liberty plays at Connecticut Sun
Day 12: Chicago Sky plays at Indiana Fever
Day 12: New York Liberty plays at Connecticut Sun
Day 13: Chicago Sky plays at Indiana Fever
Day 13: New York Liberty plays at Connecticut Sun
Day 14: Chicago Sky plays at Indiana Fever
Day 14: New York Liberty plays at Connecticut Sun
Day 15: Chicago Sky plays at Indiana Fever
Day 15: New York Liberty plays at Connecticut Sun
Day 16: Chicago Sky plays at Indiana Fever
Day 17: Atlanta Dream plays at Indiana Fever
Day 17: New York Liberty plays at Connecticut Sun
Day 18: Atlanta Dream plays at Indiana Fever
Day 18: New York Liberty plays at Connecticu

In [4]:
println("Status: ", termination_status(model))
println("Primal status: ", primal_status(model))
println("Dual status: ", dual_status(model))

Status: OPTIMAL
Primal status: FEASIBLE_POINT
Dual status: NO_SOLUTION


In [6]:
println(objective_value(model))

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [ ]:
distances = CSV.read("wnba_distances.csv", DataFrame)[:, 2:end]

teams = ["Dream", "Sky", "Sun", "Fever", 
"Liberty", "Mystics", "Wings", "Aces", 
"Sparks", "Lynx", "Mercury", "Storm"
]
cities = ["Atlanta", "Chicago", "Connecticut", 
"Indiana", "New York", "Washington", "Dallas", 
"Las Vegas", "Los Angeles", "Minnesota", "Phoenix", "Seattle"
]


home_city = Dict(team => city for team in teams, city in cities)

n = length(teams)

conferences = Dict(
    "Eastern" => ["Dream", "Sky", "Sun", "Fever", 
        "Liberty", "Mystics"],
    "Western" => ["Wings", "Aces", 
        "Sparks", "Lynx", "Mercury", "Storm"]
)

team_indices = Dict(team => i for (i, team) in enumerate(teams))

city_indices = Dict(city => i for (i, city) in enumerate(cities))

conference_indices = Dict(conf => [team_indices[team] for team in teams] for (conf, teams) in conferences)

distances = select(distances, names(distances) .=> cities)

distances = Dict((i,j) => distances[city_indices[i], city_indices[j]] for i in cities, j in cities if i != j)

In [ ]:
# # Parameters
# T = 128  # Number of days
# home_games = 20
# away_games = 20

# # Model
# model = Model(Gurobi.Optimizer)

# # Decision variables
# @variable(model, x[teams, cities, 1:T], Bin)  # 1 if a team is in a city at a time
# @variable(model, y[teams, teams, 1:T], Bin)  # 1 if teams i and j are playing each other at time t
# @variable(model, d[teams] >= 0)              # Total distance traveled by each team

# # Objective: Minimize total travel distance
# @objective(model, Min, sum(d[i] for i in teams))

# # Constraints

# # 1. Each team must be in exactly one city at each time
# @constraint(model, [i in teams, t in 1:T], sum(x[i, j, t] for j in cities) == 1)

# # 2. Home game constraints: 20 home games per team
# @constraint(model, [i in teams], sum(x[i, home_city[i], t] for t in 1:T) == home_games)

# # 3. Away game constraints: 20 away games per team
# @constraint(model, [i in teams], 
#     sum(x[i, j, t] for j in cities, t in 1:T if j != home_city[i]) == away_games)

# # 4. Teams playing each other
# for i in teams, j in teams, t in 1:T
#     if i != j
#         @constraint(model, y[i, j, t] <= x[i, home_city[j], t] + x[j, home_city[i], t])
#         @constraint(model, y[i, j, t] >= x[i, home_city[j], t] + x[j, home_city[i], t] - 1)
#         @constraint(model, y[i, j, t] == y[j, i, t])  # Symmetry constraint
#     end
# end

# # 5. Travel continuity: Distance calculation
# for i in teams, t in 1:T-1
#     for j in cities, k in cities
#         if j != k
#             @constraint(model, d[i] >= distances[(j, k)] * (x[i, j, t] + x[i, k, t+1] - 1))
#         end
#     end
# end

# # Solve the model
# optimize!(model)

# # Results
# if termination_status(model) == MOI.OPTIMAL
#     println("Optimal travel distances: ", value.(d))
#     println("Schedule: ", value.(x))
# else
#     println("No optimal solution found.")
#     println(termination_status(model))
# end

LoadError: UndefVarError: `Gurobi` not defined

In [ ]:
# # Parameters
# T = 128  # Number of days
# home_games = 20
# away_games = 20

# # Model
# model = Model(Gurobi.Optimizer)

# @variable(model, y[teams, cities, 1:T], Bin) # 1 if team has a game in city
# @variable(model, x[teams, cities, 1:T], Bin)  # 1 if a team is in a city at a time
# @variable(model, travel_distance[teams, 1:T-1] >= 0)  # Track daily travel distance
# @variable(model, d[teams] >= 0)              # Total distance traveled by each team

# # Objective: Minimize total travel distance
# @objective(model, Min, sum(d[i] for i in teams))

# # 1. Each team must be in exactly one city at each time
# @constraint(model, [i in teams, t in 1:T], sum(x[i, j, t] for j in cities) == 1)

# # 2. Home game constraints: 20 home games per team
# @constraint(model, [i in teams], sum(y[i, home_city[i], t] for t in 1:T) == home_games)

# # 3. Away game constraints: 20 away games per team
# @constraint(model, [i in teams], 
#     sum(y[i, j, t] for j in cities, t in 1:T if j != home_city[i]) == away_games)

# # team must be in a city to play a game there
# for i in teams, j in cities, t in 1:T
#     @constraint(model, y[i, j, t] <= x[i, j, t])
# end

# # both teams need to count the game
# for i in teams, j in teams, t in 1:T
#     if i != j
#         @constraint(model, y[i, home_city[j], t] == y[j, home_city[j], t])
#     end
# end

# for i in teams, t in 1:T-1
#     @constraint(model, travel_distance[i, t] >= 
#         sum(distances[(j, k)] * x[i, j, t] * x[i, k, t+1] for j in cities, k in cities if j != k))
# end

# @constraint(model, [i in teams], d[i] == sum(travel_distance[i, t] for t in 1:T-1))

# # Solve the model
# optimize!(model)

# # Results
# if termination_status(model) == MOI.OPTIMAL
#     println("Optimal travel distances: ", value.(d))
#     println("Schedule: ", value.(y))
# else
#     println("No optimal solution found.")
# end

# println(termination_status(model))

In [19]:
output = value.(y)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, ["Dream", "Sky", "Sun", "Fever", "Liberty", "Mystics", "Wings", "Aces", "Sparks", "Lynx", "Mercury", "Storm"]
    Dimension 2, ["Atlanta", "Chicago", "Connecticut", "Indiana", "New York", "Washington", "Dallas", "Las Vegas", "Los Angeles", "Minnesota", "Phoenix", "Seattle"]
    Dimension 3, Base.OneTo(128)
And data, a 12×12×128 Array{Float64, 3}:
[:, :, 1] =
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.